In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd
from scipy.stats import norm

import sys
sys.path.append("../")
from bargains_linear import * 

In [2]:
def solve_c1_c2(beta1,lambda_val,phi1,phi2):
    return (3.0*beta1*lambda_val + 2.0*beta1*phi2 - 3.0*lambda_val + 2.0*phi1 - 2.0*phi2) / (beta1 - 1.0) 

def solve_beta1(c1_c2, lambda_val,phi1,phi2):
    return (3.0*lambda_val - 2.0*phi1 + 2.0*phi2 - c1_c2 )/(3.0*lambda_val + 2.0*phi2 - c1_c2)

def solve_foc(beta1, c1_c2, lambda_val, phi1, phi2):
    return beta1*(3.0*lambda_val + 2.0*phi2 - c1_c2) - (3.0*lambda_val - 2.0*phi1 + 2.0*phi2 - c1_c2 )

In [7]:
def calc_foc_se(beta1, c1_c2, lambda_val, phi1, phi2):
    cov = np.cov(np.concatenate([phi1, phi2]))
    gradient = np.array([-2, 2*beta1-2])
    se = gradient.T.dot(cov).dot(gradient)
    return se


def perturb_c(df, c1_c2_obs,shift):
    # Create a new dataframe with d1 and d2
    new_df = df.copy()[['c1','c2','phi1','phi2']]
    new_df['c1'] = new_df['c1'] + shift
    
    model = sm.OLS( c1_c2_obs, sm.add_constant( df[['c1','c2']] ) )
    model_fit = model.fit()
    new_df['c1-c2'] = model_fit.predict( sm.add_constant(new_df[['c1','c2']]) )
    new_df['resid'] = model_fit.resid
    df['resid'] = model_fit.resid
    
    for col in ['phi1','phi2']:
        model = sm.OLS(df[col], sm.add_constant( df['c1-c2'] - new_df['resid']  ) )
        model_fit = model.fit()
        new_phi = model_fit.predict( sm.add_constant(new_df['c1-c2']+new_df['resid']) )
        new_df[col] = new_phi 

    return new_df

def calc_beta_se(c1_c2, lambda_val,phi1,phi2):
    num = (3.0*lambda_val - 2.0*phi1 + 2.0*phi2 - c1_c2 )
    denom =  (3.0*lambda_val + 2.0*phi2 - c1_c2)
    cov =np.cov(np.concatenate([num, denom]))
    gradient = np.array([1, -1/(denom.mean())**2 ])
    se = gradient.T.dot(cov).dot(gradient)
    return se


def create_test_stat(df,c1_c2=0):
    df = df.copy()
    df = df[ (df['s1']!=0) & (df['s1']!=1) &  (df['phi1']>=0) & (df['phi2']>=0) & (np.abs(df['phi1'])<=100) & (np.abs(df['phi2'])<=100) ]
    df = df.reindex()
    
    lambda_val = ( df['p1'] - (2*df['phi1'] + df['phi2'])/3 ).mean()
    
    beta1 = solve_beta1(c1_c2, lambda_val,df['phi1'].mean(),df['phi2'].mean())
    beta2 = solve_beta1(c1_c2, lambda_val,df['phi2'].mean(),df['phi1'].mean())
    
    c1_c2_obs1 = solve_c1_c2(beta1,lambda_val,df['phi1'],df['phi2'])
    c1_c2_obs2 = solve_c1_c2(beta2,lambda_val,df['phi2'],df['phi1'])
    
    (np.array(list(c1_c2_obs1)) + np.array(list(c1_c2_obs2))).var()
    argmax = (c1_c2_obs1 + c1_c2_obs2).argmax()
    return  (np.array(list(c1_c2_obs1)) + np.array(list(c1_c2_obs2))).var()
    
    
    
    
passive_result1 = pd.read_csv('../GMM_est/fake_data/passive_data_1.csv',index_col=0)
active_result1 = pd.read_csv('../GMM_est/fake_data/active_data_1.csv',index_col=0)
seq_result1 = pd.read_csv('../GMM_est/fake_data/seq_data_1.csv',index_col=0)
for df in [passive_result1,active_result1,seq_result1]:
    print(create_test_stat(df.sample(frac=0.1)))
    print('--')
print('--------------------')


-17.040686509151385 30.526851894183974
2.401243573701393
--
-32.74062819241809 42.821455138258614
3.061178760261465
--
29.992588279266403 -5.654271075590193
190.85091042602136
--
--------------------


In [4]:
passive_result1 = pd.read_csv('../GMM_est/fake_data/passive_data_3.csv',index_col=0)
active_result1 = pd.read_csv('../GMM_est/fake_data/active_data_3.csv',index_col=0)
seq_result1 = pd.read_csv('../GMM_est/fake_data/seq_data_3.csv',index_col=0)
for df in [passive_result1,active_result1,seq_result1]:
    print(create_test_stat(df.sample(frac=0.1)))
    print('--') 

7.352035567886136
--
14.33917545823216
--
80.36842641113799
--
